# 1. Setup

In [2]:
!pip install requests

You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Run a local elasticsearch
https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html#docker-cli-run-dev-mode


In [4]:
!docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.4.1

bf1b1f3476d43fa5bba344ccb1c4ca7e3a460c39a1c0afeba450a4a896660e31


In [12]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200/'); res.json()

{'name': 'bf1b1f3476d4',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'VXP1wumQREK_wBqnmCluQw',
 'version': {'number': '7.4.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'fc0eeb6e2c25915d63d871d344e3d0b45ea0ea1e',
  'build_date': '2019-10-22T17:16:35.176724Z',
  'build_snapshot': False,
  'lucene_version': '8.2.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### Install dependencies

In [13]:
!pip install elasticsearch pandas

You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.


In [14]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
pd.set_option('display.max_colwidth', 500) 
pd.set_option('display.max_columns', None)

### Create elastic client

Please use TLS for other than local dev!

In [15]:
es_client = Elasticsearch('http://localhost:9200')

# 2. Get and load some data

### Load dataset

In [16]:
# Dataset of products sold on some selected ecommerce websites (e.g. best buy)
url = "https://raw.githubusercontent.com/algolia/datasets/master/ecommerce/records.json"
df = pd.read_json(url).set_index('objectID').rename(columns={"name": "product_name"}); len(df)

10000

In [17]:
df.columns

Index(['product_name', 'description', 'brand', 'categories',
       'hierarchicalCategories', 'type', 'price', 'price_range', 'image',
       'url', 'free_shipping', 'popularity', 'rating'],
      dtype='object')

In [18]:
df.head(2)

,product_name,description,brand,categories,hierarchicalCategories,type,price,price_range,image,url,free_shipping,popularity,rating
objectID,,,,,,,,,,,,,
1696302,3-Year Unlimited Cloud Storage Service Activation Card - Other,"Enjoy 3 years of unlimited Cloud storage service with this activation card, which allows you to remotely access your favorite music, movies and other media via a compatible device and enables private file sharing with loved ones.",Pogoplug,"[Best Buy Gift Cards, Entertainment Gift Cards]","{'lvl0': 'Best Buy Gift Cards', 'lvl1': 'Best Buy Gift Cards > Entertainment Gift Cards'}",Online data backup,69.00,50 - 100,https://cdn-demo.algolia.com/bestbuy/1696302_sc.jpg,http://www.bestbuy.com/site/3-year-unlimited-cloud-storage-service-activation-card-other/1696302.p?id=1219066776306&skuId=1696302&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,True,10000,2
9131042,360fly - Panoramic 360° HD Video Camera - Black,"Only at Best Buy This 360fly panoramic 360&#176; HD video camera features Wi-Fi, a Bluetooth LE interface and the included 360fly app (download required), so you can easily shoot, edit, replay and share recorded video footage using a compatible iOS or Android smartphone. Included action camera adaptor lets you use your 360fly camera with your existing 3-prong mounts.",360fly,"[Cameras & Camcorders, Digital Cameras, Point & Shoot Cameras, 360 & Panoramic Cameras]","{'lvl0': 'Cameras & Camcorders', 'lvl1': 'Cameras & Camcorders > Digital Cameras', 'lvl2': 'Cameras & Camcorders > Digital Cameras > Point & Shoot Cameras', 'lvl3': 'Cameras & Camcorders > Digital Cameras > Point & Shoot Cameras > 360 & Panoramic Cameras'}",Point&#38;shoot camcrder,399.99,200 - 500,https://cdn-demo.algolia.com/bestbuy/9131042_rb.jpg,http://www.bestbuy.com/site/360fly-panoramic-360-hd-video-camera-black/9131042.p?id=1219405459683&skuId=9131042&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9999,5


### Load Data to Elasticsearch

**Elasticsarch Documents:**

* _index:    Indicating to which index the documents belong 
* _id:       Unique ID of your document (useful to easily update / remove)
* _source:   Contains the fields of your document

In [19]:
def doc_generator(df, es_index):
    return ({
            "_index": es_index,
            "_id": idx,
            "_source": es_record.to_dict(),
    } for idx, es_record in df.iterrows())

Loading data in bulk to Elasticsearch

In [20]:
def load_data(es_client, es_index, df):
    return helpers.bulk(
        client=es_client,
        actions=doc_generator(df, es_index=es_index)
    )

In [21]:
load_data(es_client, 'test_index', df)

(10000, [])

# 2. Query Elasticsearch

Many different query are provided in Elasticsearch. Let's start by retrieving a single document:

In [22]:
# Query 1 of the documents
import json

print(json.dumps(
    es_client.get(index='test_index', id='1696302'),
    indent=2)
 )

{
  "_index": "test_index",
  "_type": "_doc",
  "_id": "1696302",
  "_version": 1,
  "_seq_no": 0,
  "_primary_term": 1,
  "found": true,
  "_source": {
    "product_name": "3-Year Unlimited Cloud Storage Service Activation Card - Other",
    "description": "Enjoy 3 years of unlimited Cloud storage service with this activation card, which allows you to remotely access your favorite music, movies and other media via a compatible device and enables private file sharing with loved ones.",
    "brand": "Pogoplug",
    "categories": [
      "Best Buy Gift Cards",
      "Entertainment Gift Cards"
    ],
    "hierarchicalCategories": {
      "lvl0": "Best Buy Gift Cards",
      "lvl1": "Best Buy Gift Cards > Entertainment Gift Cards"
    },
    "type": "Online data backup",
    "price": 69.0,
    "price_range": "50 - 100",
    "image": "https://cdn-demo.algolia.com/bestbuy/1696302_sc.jpg",
    "url": "http://www.bestbuy.com/site/3-year-unlimited-cloud-storage-service-activation-card-other/16

### The "query_string"

Using the search API of Elasticsearch (https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html)

In [23]:
def query_elasticsearch(index, query):
    return es_client.search(
        index=index,
        body=query
    )

Returns documents based on a provided query string

In [24]:
def basic_query_string(search_terms):
    return {
        "query": {
            "query_string" : {
                "query" : search_terms
            }
        }
    }

Let's try a simple query

In [25]:
resp = query_elasticsearch("test_index", basic_query_string("car audio"))

In [26]:
resp

{'took': 231,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1340, 'relation': 'eq'},
  'max_score': 12.235233,
  'hits': [{'_index': 'test_index',
    '_type': '_doc',
    '_id': '3500126',
    '_score': 12.235233,
    '_source': {'product_name': 'Kenwood - CD - Built-in Bluetooth - Built-In HD Radio - In-Dash Deck with Detachable Faceplate - Variable Color',
     'description': 'Enjoy your favorite music with this Kenwood KDC-BT762HD in-dash deck, which features CD and MP3 playback, plus Apple&#174; iPhone&#174; and iPod&#174; and Android connectivity for versatile playback options. Built-in Bluetooth enables wireless streaming.',
     'brand': 'Kenwood',
     'categories': ['Car Electronics & GPS',
      'Car Audio',
      'Car Stereo Receivers',
      'In-Dash with HD Radio',
      'Built-In HD Radio Decks'],
     'hierarchicalCategories': {'lvl0': 'Car Electronics & GPS',
      'lvl1': 'Car Electronics & GPS 

Let's build a small function to display nicely the JSON obtain in response

In [27]:
from pandas.io.json import json_normalize

def display_results(resp):
    return json_normalize(resp['hits']['hits'])

In [28]:
display_results(resp)[:5]

,_index,_type,_id,_score,_source.product_name,_source.description,_source.brand,_source.categories,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.hierarchicalCategories.lvl4,_source.type,_source.price,_source.price_range,_source.image,_source.url,_source.free_shipping,_source.popularity,_source.rating
0,test_index,_doc,3500126,12.235233,Kenwood - CD - Built-in Bluetooth - Built-In HD Radio - In-Dash Deck with Detachable Faceplate - Variable Color,"Enjoy your favorite music with this Kenwood KDC-BT762HD in-dash deck, which features CD and MP3 playback, plus Apple&#174; iPhone&#174; and iPod&#174; and Android connectivity for versatile playback options. Built-in Bluetooth enables wireless streaming.",Kenwood,"[Car Electronics & GPS, Car Audio, Car Stereo Receivers, In-Dash with HD Radio, Built-In HD Radio Decks]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Stereo Receivers,Car Electronics & GPS > Car Audio > Car Stereo Receivers > In-Dash with HD Radio,Car Electronics & GPS > Car Audio > Car Stereo Receivers > In-Dash with HD Radio > Built-In HD Radio Decks,Cd decks,169.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/3500126_sb.jpg,http://www.bestbuy.com/site/kenwood-cd-built-in-bluetooth-built-in-hd-radio-in-dash-deck-with-detachable-faceplate-variable-color/3500126.p?id=1219582881375&skuId=3500126&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,4712,3
1,test_index,_doc,9225581,9.593058,Griffin Technology - StereoConnect Cable for Most Portable Audio Devices,Easily connect a portable audio device to your home or car stereo system with this convenient coiled cable.,Griffin Technology,"[Audio, Cables & Chargers, MP3 Player Cables]",Audio,Audio > Cables & Chargers,Audio > Cables & Chargers > MP3 Player Cables,NaN,NaN,Connectivity,22.99,1 - 50,https://cdn-demo.algolia.com/bestbuy/9225581_sb.jpg,http://www.bestbuy.com/site/griffin-technology-stereoconnect-cable-for-most-portable-audio-devices/9225581.p?id=1218062421800&skuId=9225581&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,6413,5
2,test_index,_doc,7551355,8.856272,"Alpine - Type-S 5-1/4"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black","Upgrade your car audio system with these Alpine Type-S SPS-510 2-way coaxial car speakers, which handle up to 170W peak power and feature 5-1/4"" poly-mica woofer cones and 3/4"" silk-dome tweeters for a wide range of frequencies.",Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 5.25"" Car Speakers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 5.25"" Car Speakers",NaN,Speakers,89.99,50 - 100,https://cdn-demo.algolia.com/bestbuy/7551355_sb.jpg,http://www.bestbuy.com/site/alpine-type-s-5-1-4-2-way-coaxial-car-speakers-with-poly-mica-cones-pair-black/7551355.p?id=1218853533848&skuId=7551355&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9899,5
3,test_index,_doc,5886113,8.658096,Insignia™ - 6' Mini Stereo Audio Cable - White,"Connect your portable audio device to a 3.5mm auxiliary stereo audio input with this Insignia&#8482; NS-PO35506 mini stereo cable, so you can enjoy rich stereo sound through your compatible car stereo, portable speakers, computer or TV.",Insignia™,"[Computers & Tablets, Cables & Connectors]",Computers & Tablets,Computers & Tablets > Cables & Connectors,NaN,NaN,NaN,Cables/connectors,9.99,1 - 50,https://cdn-demo.algolia.com/bestbuy/5886113_sb.jpg,http://www.bestbuy.com/site/insignia-6-mini-stereo-audio-cable-white/5886113.p?id=1219169458938&skuId=5886113&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,5561,3
4,test_index,_doc,7576083,8.506055,Alpine - CD - Built-In Bluetooth - In-Dash Deck with Satellite Radio Tuner - Black,Enhance the audio system in your car with this Alpine in-dash CD deck that is SiriusXM Satellite Radi

The results are showing some audio cables that have little to do with a car ...

### Field Boosting

We can boost specific field in our query (https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-boost.html)

In [49]:
# Give higher weight to categories
def query_with_weight(search_terms):
    return {
        "query": {
            "query_string" : {
                "query" : search_terms,
                "fields": [
                    "description^2",
                    "categories^3",
                    "product_name^2",
                    "brand",
                    "type"
                ]
            }
        }
    }

resp = query_elasticsearch("test_index", query_with_weight("car audio"))
display_results(resp)[:2]

,_index,_type,_id,_score,_source.product_name,_source.description,_source.brand,_source.categories,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.type,_source.price,_source.price_range,_source.image,_source.url,_source.free_shipping,_source.popularity,_source.rating
0,test_index,_doc,4769785,19.982872,"Alpine - 12"" Dual-Voice-Coil 4-Ohm Subwoofer - Black","Listen to music in your vehicle with rich, clear sound with this Alpine SWS-12D4 12"" dual-voice-coil 4-ohm subwoofer that features improved thermal capacity and mechanical strength for reliable performance.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,149.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/4769785_sb.jpg,http://www.bestbuy.com/site/alpine-12-dual-voice-coil-4-ohm-subwoofer-black/4769785.p?id=1218524494220&skuId=4769785&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9915,3
1,test_index,_doc,9597106,19.982872,"Alpine - 8"" Powered Subwoofer System - Black","Amplify the bass in your vehicle using this Alpine PWE-S8 subwoofer system that features a compact design you can fit beneath or behind a truck seat, in a sedan trunk or in the hatchback of an SUV. The cast-metal construction ensures long-lasting use.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,299.99,200 - 500,https://cdn-demo.algolia.com/bestbuy/9597106_sb.jpg,http://www.bestbuy.com/site/alpine-8-powered-subwoofer-system-black/9597106.p?id=1219424288868&skuId=9597106&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9907,5


Better already. But what if I am looking for some subwoofer?

In [30]:
resp = query_elasticsearch("test_index", query_with_weight('car audio 170 watt'))
display_results(resp)[:6]

,_index,_type,_id,_score,_source.product_name,_source.description,_source.brand,_source.categories,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.type,_source.price,_source.price_range,_source.image,_source.url,_source.free_shipping,_source.popularity,_source.rating
0,test_index,_doc,4769785,19.302147,"Alpine - 12"" Dual-Voice-Coil 4-Ohm Subwoofer - Black","Listen to music in your vehicle with rich, clear sound with this Alpine SWS-12D4 12"" dual-voice-coil 4-ohm subwoofer that features improved thermal capacity and mechanical strength for reliable performance.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,149.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/4769785_sb.jpg,http://www.bestbuy.com/site/alpine-12-dual-voice-coil-4-ohm-subwoofer-black/4769785.p?id=1218524494220&skuId=4769785&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9915,3
1,test_index,_doc,9597106,19.302147,"Alpine - 8"" Powered Subwoofer System - Black","Amplify the bass in your vehicle using this Alpine PWE-S8 subwoofer system that features a compact design you can fit beneath or behind a truck seat, in a sedan trunk or in the hatchback of an SUV. The cast-metal construction ensures long-lasting use.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,299.99,200 - 500,https://cdn-demo.algolia.com/bestbuy/9597106_sb.jpg,http://www.bestbuy.com/site/alpine-8-powered-subwoofer-system-black/9597106.p?id=1219424288868&skuId=9597106&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9907,5
2,test_index,_doc,8815536,19.302147,"Alpine - Bassline 12"" Single-Voice-Coil 4-Ohm Subwoofer - Black","Enhance your vehicle's audio system with this Alpine Bassline SWA-12S4 single-voice-coil, 4-ohm subwoofer that handles up to 750W peak power and features a 12"" polypropylene/mica woofer cone with a UV-resistant poly foam surround for robust sound.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,74.99,50 - 100,https://cdn-demo.algolia.com/bestbuy/8815536_sb.jpg,http://www.bestbuy.com/site/alpine-bassline-12-single-voice-coil-4-ohm-subwoofer-black/8815536.p?id=1218902549936&skuId=8815536&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,True,9906,3
3,test_index,_doc,8480035,19.302147,"JBL - CX Series 12"" Single-Voice-Coil 4-Ohm Subwoofer - Black","Enjoy powerful low-end sound while you're on the road with this JBL CX Series CX1200 single-voice-coil subwoofer, which features a 12"" polypropylene woofer cone for rich, robust audio. The progressive spider delivers optimal control.",JBL,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,129.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/8480035_sb.jpg,http://www.bestbuy.com/site/jbl-cx-series-12-single-voice-coil-4-ohm-subwoofer-black/8480035.p?id=1219339620491&skuId=8480035&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,4938,6
4,test_index,_doc,3581048,19.302147,"Kenwood - 12"" 4-Ohm Subwoofer - Silver","Jam out to your favorite tunes in your vehicle with this 12"" Kenwood KFC-W3016PS subwoofer, which features an oversize diaphragm and handles up to 2000W of power (400W RMS) for intense audio. Dual ventilation helps keep the speaker operating efficiently.",Kenwood,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,119.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/3581048_sb.jpg,http://www.bestbuy.com/site/kenwood-12-4-ohm-subwoofer-silver/3581048.p?id=121959986

Unfortunately that query does not work very well :/  Let's see what we can do to fix that

# 3. Mapping (Schema)

In [31]:
def create_elasticsearch_mapping(index, mapping):
    # ignore 400 cause by IndexAlreadyExistsException when creating an index
    return es_client.indices.create(index=index, body=mapping, ignore=400)

def delete_elasticsearch_mapping(index):
    return es_client.indices.delete(index=index, ignore=[400, 404])

Each field has a data type which can be:

* a simple type like text, keyword, date, long, double, boolean or ip.
* a type which supports the hierarchical nature of JSON such as object or nested.
* a specialised type like geo_point, geo_shape, or completion.

In [32]:
es_mapping = {
    "mappings": {
        "properties": {
            "product_name":           { "type": "text" },  
            "description":            { "type": "text" },
            "categories":             { "type": "text" },
            "hierarchicalCategories": {
                "type": "nested",
                "include_in_parent": "true",
                "properties": {
                    "lvl0": {"type": "text"},
                    "lvl1": {"type": "text"},
                    "lvl2": {"type": "text"},
                    "lvl3": {"type": "text"},
                    "lvl4": {"type": "text"}
                }
            },
            "type":                   { "type": "text" },
            "brand":                  { "type": "text" },
            "url":                    { "type": "text", "index" : False },
            "image":                  { "type": "text", "index" : False },
            "popularity":             { "type": "double" },
            "price":                  { "type": "text" },
            "price_range":            { "type": "keyword" },
            "rating":                 { "type": "short" },
            "free_shipping":          { "type": "boolean" }
    }
  }
}

delete_elasticsearch_mapping('test-w-mapping')
create_elasticsearch_mapping("test-w-mapping", es_mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-w-mapping'}

In [33]:
load_data(es_client, 'test-w-mapping', df)

(10000, [])

In [34]:
resp = query_elasticsearch("test-w-mapping", query_with_weight("car audio 170 watt"))
display_results(resp)[:10]

,_index,_type,_id,_score,_source.product_name,_source.description,_source.brand,_source.categories,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.type,_source.price,_source.price_range,_source.image,_source.url,_source.free_shipping,_source.popularity,_source.rating
0,test-w-mapping,_doc,4769785,19.087585,"Alpine - 12"" Dual-Voice-Coil 4-Ohm Subwoofer - Black","Listen to music in your vehicle with rich, clear sound with this Alpine SWS-12D4 12"" dual-voice-coil 4-ohm subwoofer that features improved thermal capacity and mechanical strength for reliable performance.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,149.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/4769785_sb.jpg,http://www.bestbuy.com/site/alpine-12-dual-voice-coil-4-ohm-subwoofer-black/4769785.p?id=1218524494220&skuId=4769785&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9915,3
1,test-w-mapping,_doc,9597106,19.087585,"Alpine - 8"" Powered Subwoofer System - Black","Amplify the bass in your vehicle using this Alpine PWE-S8 subwoofer system that features a compact design you can fit beneath or behind a truck seat, in a sedan trunk or in the hatchback of an SUV. The cast-metal construction ensures long-lasting use.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,299.99,200 - 500,https://cdn-demo.algolia.com/bestbuy/9597106_sb.jpg,http://www.bestbuy.com/site/alpine-8-powered-subwoofer-system-black/9597106.p?id=1219424288868&skuId=9597106&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9907,5
2,test-w-mapping,_doc,8815536,19.087585,"Alpine - Bassline 12"" Single-Voice-Coil 4-Ohm Subwoofer - Black","Enhance your vehicle's audio system with this Alpine Bassline SWA-12S4 single-voice-coil, 4-ohm subwoofer that handles up to 750W peak power and features a 12"" polypropylene/mica woofer cone with a UV-resistant poly foam surround for robust sound.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,74.99,50 - 100,https://cdn-demo.algolia.com/bestbuy/8815536_sb.jpg,http://www.bestbuy.com/site/alpine-bassline-12-single-voice-coil-4-ohm-subwoofer-black/8815536.p?id=1218902549936&skuId=8815536&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,True,9906,3
3,test-w-mapping,_doc,8480035,19.087585,"JBL - CX Series 12"" Single-Voice-Coil 4-Ohm Subwoofer - Black","Enjoy powerful low-end sound while you're on the road with this JBL CX Series CX1200 single-voice-coil subwoofer, which features a 12"" polypropylene woofer cone for rich, robust audio. The progressive spider delivers optimal control.",JBL,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,129.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/8480035_sb.jpg,http://www.bestbuy.com/site/jbl-cx-series-12-single-voice-coil-4-ohm-subwoofer-black/8480035.p?id=1219339620491&skuId=8480035&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,4938,6
4,test-w-mapping,_doc,3581048,19.087585,"Kenwood - 12"" 4-Ohm Subwoofer - Silver","Jam out to your favorite tunes in your vehicle with this 12"" Kenwood KFC-W3016PS subwoofer, which features an oversize diaphragm and handles up to 2000W of power (400W RMS) for intense audio. Dual ventilation helps keep the speaker operating efficiently.",Kenwood,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,Subs,119.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/3581048_sb.jpg,http://www.bestbuy.com/site/kenwood-12-4-ohm-subwoofer-silver/35

Still not working ...

# 4. Analyzers

<img src="https://www.elastic.co/guide/en/elasticsearch/client/net-api/current/analysis-chain.png" alt="analyzers in elasticsearch" width="600"/>

One possible solution is to convert the char during analysis. Let's add the `char_filter` to the default `standard` analyzer (https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-standard-analyzer.html)

In [35]:
es_mapping_v2 = {
    "settings" : {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "char_filter": ["my_char_filter"],
                    "tokenizer": "standard",
                    "filter": [
                        "my_synonyms"
                    ]
                }
            },
            "char_filter": {
                "my_char_filter": {
                    "type": "pattern_replace",
                    "pattern": "([0-9]+)W",
                    "replacement": "$1 watt"
                }
            },
            "filter": {
                "my_synonyms": {
                  "type": "synonym",
                  "synonyms": [
                    "sub, subwoofer"
                  ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "product_name":           { "type": "text" },  
            "description":            { "type": "text" , "analyzer": "my_analyzer" },
            "categories":             { "type": "text" },
            "hierarchicalCategories": {
                "type": "nested",
                "include_in_parent": "true",
                "properties": {
                    "lvl0": {"type": "text"},
                    "lvl1": {"type": "text"},
                    "lvl2": {"type": "text"},
                    "lvl3": {"type": "text"},
                    "lvl4": {"type": "text"}
                }
            },
            "type":                   { "type": "text" },
            "brand":                  { "type": "text" },
            "url":                    { "type": "text", "index" : False },
            "image":                  { "type": "text", "index" : False },
            "popularity":             { "type": "double" },
            "price":                  { "type": "text" },
            "price_range":            { "type": "keyword" },
            "rating":                 { "type": "short" },
            "free_shipping":          { "type": "boolean" }
    }
  }
}

delete_elasticsearch_mapping("test-w-mapping-v2")
create_elasticsearch_mapping("test-w-mapping-v2", es_mapping_v2)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'test-w-mapping-v2'}

Let's load some data in our new index

In [36]:
load_data(es_client, 'test-w-mapping-v2', df)

(10000, [])

Let's see if that works!

In [37]:
resp = query_elasticsearch('test-w-mapping-v2', query_with_weight("car audio 170 watt"))
display_results(resp)[:10]

,_index,_type,_id,_score,_source.product_name,_source.description,_source.brand,_source.categories,_source.hierarchicalCategories.lvl0,_source.hierarchicalCategories.lvl1,_source.hierarchicalCategories.lvl2,_source.hierarchicalCategories.lvl3,_source.type,_source.price,_source.price_range,_source.image,_source.url,_source.free_shipping,_source.popularity,_source.rating
0,test-w-mapping-v2,_doc,7551355,37.180183,"Alpine - Type-S 5-1/4"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black","Upgrade your car audio system with these Alpine Type-S SPS-510 2-way coaxial car speakers, which handle up to 170W peak power and feature 5-1/4"" poly-mica woofer cones and 3/4"" silk-dome tweeters for a wide range of frequencies.",Alpine,"[Car Electronics & GPS, Car Audio, Car Speakers, 5.25"" Car Speakers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Speakers,"Car Electronics & GPS > Car Audio > Car Speakers > 5.25"" Car Speakers",Speakers,89.99,50 - 100,https://cdn-demo.algolia.com/bestbuy/7551355_sb.jpg,http://www.bestbuy.com/site/alpine-type-s-5-1-4-2-way-coaxial-car-speakers-with-poly-mica-cones-pair-black/7551355.p?id=1218853533848&skuId=7551355&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9899,5
1,test-w-mapping-v2,_doc,7752088,23.520294,"Kicker - TCompS10 10"" Single-Voice-Coil 4-Ohm Subwoofer with Enclosure - Black","Upgrade your car audio system with this Kicker TCompS10 40TCWS104 single-voice-coil subwoofer, which handles up to 600W peak power for deep bass sound and includes a ported box for simplified installation.",Kicker,"[Car Electronics & GPS, Car Audio, Loaded Enclosures]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Loaded Enclosures,NaN,Subs,149.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/7752088_sb.jpg,http://www.bestbuy.com/site/kicker-tcomps10-10-single-voice-coil-4-ohm-subwoofer-with-enclosure-black/7752088.p?id=1218859509039&skuId=7752088&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,4599,3
2,test-w-mapping-v2,_doc,4769785,21.254700,"Alpine - 12"" Dual-Voice-Coil 4-Ohm Subwoofer - Black","Listen to music in your vehicle with rich, clear sound with this Alpine SWS-12D4 12"" dual-voice-coil 4-ohm subwoofer that features improved thermal capacity and mechanical strength for reliable performance.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,NaN,Subs,149.99,100 - 200,https://cdn-demo.algolia.com/bestbuy/4769785_sb.jpg,http://www.bestbuy.com/site/alpine-12-dual-voice-coil-4-ohm-subwoofer-black/4769785.p?id=1218524494220&skuId=4769785&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9915,3
3,test-w-mapping-v2,_doc,9597106,21.254700,"Alpine - 8"" Powered Subwoofer System - Black","Amplify the bass in your vehicle using this Alpine PWE-S8 subwoofer system that features a compact design you can fit beneath or behind a truck seat, in a sedan trunk or in the hatchback of an SUV. The cast-metal construction ensures long-lasting use.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electronics & GPS > Car Audio,Car Electronics & GPS > Car Audio > Car Subwoofers,NaN,Subs,299.99,200 - 500,https://cdn-demo.algolia.com/bestbuy/9597106_sb.jpg,http://www.bestbuy.com/site/alpine-8-powered-subwoofer-system-black/9597106.p?id=1219424288868&skuId=9597106&cmp=RMX&ky=1uWSHMdQqBeVJB9cXgEke60s5EjfS6M1W,False,9907,5
4,test-w-mapping-v2,_doc,8815536,21.254700,"Alpine - Bassline 12"" Single-Voice-Coil 4-Ohm Subwoofer - Black","Enhance your vehicle's audio system with this Alpine Bassline SWA-12S4 single-voice-coil, 4-ohm subwoofer that handles up to 750W peak power and features a 12"" polypropylene/mica woofer cone with a UV-resistant poly foam surround for robust sound.",Alpine,"[Car Electronics & GPS, Car Audio, Car Subwoofers]",Car Electronics & GPS,Car Electr

Finally it works!

# Let's build a search engine!

In [45]:
def search_engine(phrase):
    resp = query_elasticsearch('test-w-mapping-v2', query_with_weight(phrase))
    
    # Get item from response
    items = [res['_source'] for res in resp['hits']['hits']]
    
    item_template = """
      <tr>
        <th style="width: 100px"><img src="{image}"></th>
        <th>{name}</th>
        <th>{brand}</th>
        <th>{description}</th>
      </tr>
    """

    page = ""
    page += """<table style="width: 70%">"""
    for row in items:
        page += item_template.format(image=row['image'], name=row['product_name'], brand=row['brand'], description=row['description'])
    page += "</table>"

    return page

In [47]:
HTML(search_engine("speaker stands"))

,Bell'O - Speaker Stands (Pair) - Black/Brown,Bell'O,Keep your speakers at the optimal height for ideal sound performance with these Bell'O SP211 speaker stands. A combination of black powdercoat steel and cherry-finish wood inlaid fronts offers a stylish accent to complement your home d&233;cor.
,Bose® - UFS-20 Series II Universal Floor Stands (Pair) - Black,Bose®,These Bose® UFS-20 Series II universal floor stands are designed to optimize listening. Slender metal stands elegantly display your speakers while hiding the speaker wires.
,Bowers & Wilkins - STAV 24 S2 Speaker Stands (Pair) - Black,Bowers & Wilkins,This pair of Bowers & Wilkins STAV 24 S2 stands accommodates your 685 S2 or 686 S2 speakers. Twin metal vertical columns provide support and help dampen vibration to maintain high-quality sound.
,Sanus - Adjustable Speaker Stands (Pair) - Black Satin,Sanus,"From the Sanus Euro Foundations line, these distinctive speaker stands are as sleek and stylish as they are functional. Stable adjustable design is ideal for satellite and surround speakers. Also available: matching TV stand;4-shelf TV and A/V stand;mini-system stand andadjustable micro speaker standsLearn more about Sanus Systems' Entertainment Furniture and Accessories."
,"Sanus - Natural 36"" Speaker Stands (Pair) - Cherry",Sanus,Learn more about Magnolia Premium Installation Enhance your home theater experience with these speaker stands that feature an attractive pillar design that improves sound quality by providing acoustic isolation. Keep wires and cables organized and neat with the convenient wire management path.
,Sanus - Speaker Stands (Pair) - Black,Sanus,"Proudly display your satellite speakers with these Sanus HTB3-B1 speaker stands, which adjust from 26"" to 39"" high for optimal acoustics and provide cord storage to conceal and reduce clutter. Rubber feet and carpet spikes ensure stability."
,Sanus - Speaker Stands for SONOS PLAY Speakers (Pair) - Black,Sanus,This Sanus WSS2-B1 stand features a multi-use bracket that lets you place your SONOS Play 1 or Play 3 speaker in a horizontal or vertical position. Rubber feet allow stable placement on hard-surface floors.
,Sanus - Speaker Stands for SONOS PLAY Speakers (Pair) - White,Sanus,Place your SONOS PLAY 1 or PLAY 3 speakers in an optimal audio position with these Sanus WSS2-W1 speaker stands that feature a wire channel to keep cables out of sight for a sleek look. The rubber feet help minimize sliding on hard floors.
,Sanus - Steel Foundations Series Bookshelf Speaker Stands (Pair) - Black,Sanus,Learn more about Magnolia Premium Installation These speaker stands feature adjustable carpet spikes and rubber feet that provide multiple stabilizing options and an integrated wire path that simplifies organization of heavy-gauge cables.
,"Sanus - Natural 18"" Speaker Stands (Pair) - Black",Sanus,Learn more about Magnolia Premium Installation Display your speakers in style with these solid hardwood pillar speaker stands that provide acoustic isolation and a wire management path for neatly storing wires and cables.


In [ ]:
{
    "settings" : {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "char_filter": ["my_char_filter"],
                    "tokenizer": "standard",
                },
            },
            "char_filter": {
                "my_char_filter": {
                    "type": "pattern_replace",
                    "pattern": "([0-9]+)W",
                    "replacement": "$1 watt"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "product_name": {"type": "text"},  
            "description":  {"type": "text", "analyzer":"my_analyzer"},
            "categories":   { "type": "text" },
            "type":         { "type": "text" },
            "brand":        { "type": "text" },
            "popularity":   { "type": "double"}
        }
    }
}